In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
import scipy.stats as sta
import warnings
warnings.filterwarnings('ignore')

In [2]:
#--train and test data--
train_df = pd.read_csv('D:/DataAnalysis/2019未来高校AI挑战赛_城市-房产租金预测/train_data.csv')
test_df = pd.read_csv('D:/DataAnalysis/2019未来高校AI挑战赛_城市-房产租金预测/test_a.csv')

In [3]:
train_df.shape

(41440, 51)

In [4]:
def proprecessing(train_df,test_df):
    
    ## fill with null values
    fill_na = ["uv","pv"]
    for fill in fill_na:
        train_df[fill].fillna(train_df[fill].mean(),inplace=True)
        test_df[fill].fillna(test_df[fill].mean(),inplace=True)
        train_df[fill] = train_df[fill].astype(int)
        test_df[fill] = test_df[fill].astype(int)
        
    ## 使用孤立森林来检测tradeMoney的异常值点
    from sklearn.ensemble import IsolationForest
    IForest = IsolationForest(contamination=0.02)
    IForest.fit(train_df['tradeMoney'].values.reshape(-1,1))
    ypred = IForest.predict(train_df['tradeMoney'].values.reshape(-1,1))
    drop_index = train_df[ypred==-1].index
    train_df.drop(drop_index,axis=0,inplace=True)
    
    ##--area--
    train_df = train_df[(train_df['area']>=10)&(train_df['area']<=200)]
    
    ##组合特征进行筛选
    #list(train_df[['region','tradeMoney','area']].groupby('region'))
    train_df.drop(train_df[(train_df['region']=='RG00001')&(train_df['area']>=100)&
                       (train_df['tradeMoney']<=2000)].index,inplace=True)
    # train_df.drop(train_df[(train_df['region']=='RG00001')&(train_df['area']<=10)&
    #                     (train_df['tradeMoney']>=3000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00002')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<=1000)].index,inplace=True)
    # train_df.drop(train_df[(train_df['region']=='RG00003')&(train_df['area']<=10)&
    #                        (train_df['tradeMoney']>=4000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00003')&(train_df['area']>=150)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00004')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<=1000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00005')&(train_df['area']>=150)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00006')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<1000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00008')&(train_df['area']>100)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00009')&(train_df['area']>=100)&
                           (train_df['tradeMoney']<=1000)].index,inplace=True)
    # train_df.drop(train_df[(train_df['region']=='RG00010')&(train_df['area']>=100)&
    #                        (train_df['tradeMoney']<=2000)].index,inplace=True)
    train_df.drop(train_df[(train_df['region']=='RG00011')&(train_df['area']>=150)&
                           (train_df['tradeMoney']<=2000)].index,inplace=True)
    
    data_df = pd.concat([train_df,test_df])
    data_df.drop(['city','ID','tradeMoney'],axis=1,inplace=True)
    
    #--rentType--
    data_df['rentType'].replace('--','未知方式',inplace=True)

    #--buildYear--
    median_build = data_df.loc[data_df['buildYear']!='暂无信息','buildYear'].median()
    data_df['buildYear'].replace("暂无信息",median_build,inplace=True)
    data_df['buildYear'] = data_df["buildYear"].astype(int)
    #data_df["buildYear"] = 2019-data_df["buildYear"]

    #--houseType--
    data_df['rooms'] = data_df['houseType'].apply(lambda x:x.split('室')[0][-1]).astype(int)
    data_df['halls'] = data_df['houseType'].apply(lambda x:x.split('厅')[0][-1]).astype(int)
    data_df['bathrooms'] = data_df['houseType'].apply(lambda x:x.split('卫')[0][-1]).astype(int)
    data_df['rooms_bathrooms'] = (data_df['bathrooms']+1)/(data_df['rooms']+1)
    data_df['rooms_bathrooms'] = data_df['rooms_bathrooms'].astype(int)
    data_df.drop('houseType',axis=1,inplace=True)
    
    #--tradeTime--
    data_df['Month'] = data_df['tradeTime'].apply(lambda x:x.split('/')[1]).astype(int)
    data_df['Date'] = data_df['tradeTime'].apply(lambda x:x.split('/')[2]).astype(int) 
    data_df.drop('tradeTime',axis=1,inplace=True)
    
    return train_df,data_df

In [5]:
train_df,data_df = proprecessing(train_df,test_df)

D:\AnacondaSoftware\Anaconda\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [6]:
# train_df['interSchoolNum'].describe()

In [7]:
# train_df['privateSchoolNum'].describe()

In [8]:
# train_df['schoolNum'].describe()

## 关于数据的分析，前2个脚本已经很详细了，这里重点在于特征的处理上

In [9]:
numerical_col = [col for col in data_df.columns if data_df[col].dtypes!='object']
numerical_col

['area',
 'bankNum',
 'buildYear',
 'busStationNum',
 'drugStoreNum',
 'gymNum',
 'hospitalNum',
 'interSchoolNum',
 'landMeanPrice',
 'landTotalPrice',
 'lookNum',
 'mallNum',
 'newWorkers',
 'parkNum',
 'privateSchoolNum',
 'pv',
 'remainNewNum',
 'residentPopulation',
 'saleSecHouseNum',
 'schoolNum',
 'shopNum',
 'subwayStationNum',
 'superMarketNum',
 'supplyLandArea',
 'supplyLandNum',
 'supplyNewNum',
 'totalFloor',
 'totalNewTradeArea',
 'totalNewTradeMoney',
 'totalTradeArea',
 'totalTradeMoney',
 'totalWorkers',
 'tradeLandArea',
 'tradeLandNum',
 'tradeMeanPrice',
 'tradeNewMeanPrice',
 'tradeNewNum',
 'tradeSecNum',
 'uv',
 'rooms',
 'halls',
 'bathrooms',
 'rooms_bathrooms',
 'Month',
 'Date']

In [10]:
##--Supporting facilities--
data_df['All_School'] = 2*data_df['interSchoolNum']/data_df['interSchoolNum'].mean() + (
    data_df['privateSchoolNum']/data_df['privateSchoolNum'].mean()+
    data_df['schoolNum']/data_df['schoolNum'].mean())
data_df['Transportation'] = 5*data_df['subwayStationNum']/data_df['subwayStationNum'].mean() + data_df['busStationNum']/data_df['busStationNum'].mean()
data_df['All_hospital'] = 3*data_df['hospitalNum']/data_df['hospitalNum'].mean()+data_df['drugStoreNum']/data_df['drugStoreNum'].mean()
data_df['All_mall'] = data_df['mallNum']/data_df['mallNum'].mean()+data_df['superMarketNum']/data_df['superMarketNum'].mean()
data_df['other_num'] = data_df['bankNum']/data_df['bankNum'].mean() + (
    data_df['gymNum']/data_df['gymNum'].mean()+data_df['parkNum']/data_df['parkNum'].mean()
    +data_df['shopNum']/data_df['shopNum'].mean())
data_df.drop(['subwayStationNum', 'busStationNum',
               'interSchoolNum', 'schoolNum', 'privateSchoolNum',
               'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 
              'gymNum', 'bankNum', 'shopNum', 'parkNum'],axis=1, inplace=True)

In [11]:
# pd.concat([data_df['landTotalPrice'],data_df['tradeLandArea']],axis=1)

In [12]:
#--Land--
data_df['SecMeanArea'] = data_df['totalTradeArea']/data_df['tradeSecNum']
data_df['SecMeanArea'].replace(np.inf,0,inplace=True)
data_df['trade_supplyNew_rate'] = data_df['tradeNewNum']/data_df['supplyNewNum']
data_df['trade_supplyNew_rate'].replace(np.inf,0,inplace=True)
data_df['remain_supply_rate'] = data_df['remainNewNum']/data_df['supplyNewNum']
data_df['remain_supply_rate'].replace(np.inf,0,inplace=True)
data_df['trade_supplyLand_rate'] = data_df['tradeLandArea']/data_df['supplyLandArea']
data_df['trade_supplyLand_rate'].fillna(0,inplace=True)

In [13]:
##--group by--
inter = data_df.groupby('communityName')['area'].agg({'com_area_mean':'mean','com_area_std':'std'})
inter.fillna(0,inplace=True)
data_df = data_df.merge(inter,on='communityName',how='left')

# data_df['com_area_mean_1'] = data_df.groupby('communityName')['area'].transform('mean')

inter = data_df.groupby('plate')['area'].agg({'pla_are_mean':'mean','pla_are_std':'std'})
data_df = data_df.merge(inter,on='plate',how='left')
#--buildYear--
inter = data_df.groupby('plate')['buildYear'].agg({'pla_build_mean':'mean','pla_build_std':'std'})
data_df = data_df.merge(inter,on='plate',how='left')
data_df.pla_build_mean = data_df.pla_build_mean.astype(int)
data_df['pla_build_diff'] = data_df.buildYear - data_df.pla_build_mean
#--school--
data_df['plate_school'] = data_df.groupby('plate')['All_School'].transform('sum')
data_df['commit_plate_school'] = data_df.groupby(['communityName','plate'])['All_School'].transform('sum')
data_df['school_ratio'] = data_df['commit_plate_school']/data_df['plate_school']
data_df['school_ratio'].replace(np.inf,-1,inplace=True)
#--Transportation--
data_df['plate_Transp'] = data_df.groupby('plate')['Transportation'].transform('sum')
data_df['commit_plate_Transp'] = data_df.groupby(['communityName','plate'])['Transportation'].transform('sum')
data_df['Transportation_ratio'] = data_df['commit_plate_Transp']/data_df['plate_Transp']
data_df['Transportation_ratio'].replace(np.inf,-1,inplace=True)
#--hospital--
data_df['plate_hospital'] = data_df.groupby('plate')['All_hospital'].transform('sum')
data_df['commit_plate_hospital'] = data_df.groupby(['communityName','plate'])['All_hospital'].transform('sum')
data_df['School_ratio'] = data_df['commit_plate_hospital']/data_df['plate_hospital']
data_df['School_ratio'].replace(np.inf,-1,inplace=True)
#--mall--
data_df['plate_mall'] = data_df.groupby('plate')['All_mall'].transform('sum')
data_df['commit_plate_mall'] = data_df.groupby(['communityName','plate'])['All_mall'].transform('sum')
data_df['mall_ratio'] = data_df['commit_plate_mall']/data_df['plate_mall']
data_df['mall_ratio'].replace(np.inf,-1,inplace=True)
#--other_num--
data_df['plate_other_num'] = data_df.groupby('plate')['other_num'].transform('sum')
data_df['commit_plate_other'] = data_df.groupby(['communityName','plate'])['other_num'].transform('sum')
data_df['other_ratio'] = data_df['commit_plate_other']/data_df['plate_other_num']
data_df['other_ratio'].replace(np.inf,-1,inplace=True)
data_df.drop(['pla_build_mean','plate_school','commit_plate_school','plate_Transp','commit_plate_Transp',
              'plate_hospital','commit_plate_hospital','plate_mall','commit_plate_mall','plate_other_num',
             'commit_plate_other'],axis=1,inplace=True)

## Cluster聚类分析

In [14]:
from sklearn.preprocessing import LabelEncoder

Label_list = ['plate','region','houseToward','communityName','houseDecoration','rentType','houseFloor']
for col_4 in Label_list:
    data_df[col_4] = data_df[col_4].astype(str)
    data_df[col_4] = LabelEncoder().fit_transform(data_df[col_4])
    
# Onehot_list = ['houseDecoration','rentType','houseFloor']
# for col_5 in Onehot_list:
#     data_df[col_5] = data_df[col_5].astype(str)
#     inter_col = pd.get_dummies(data_df[col_5])
#     data_df.drop(col_5,axis=1,inplace=True)
#     data_df = pd.concat([data_df,inter_col],axis=1)

In [16]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=5,covariance_type='full',random_state=2019)
cluster_col = ['communityName','buildYear','plate','region','landMeanPrice','lookNum','newWorkers',
               'residentPopulation','totalFloor','houseDecoration','tradeMeanPrice','tradeSecNum',
              'totalNewTradeMoney','totalNewTradeArea','tradeNewMeanPrice','tradeNewNum','remainNewNum',
              'landTotalPrice','landMeanPrice','totalWorkers','All_School','Transportation','All_hospital',
              'All_mall','other_num']
data_df['cluster'] = gmm.fit_predict(data_df[cluster_col])

In [17]:
cluster_col_2 = ['totalFloor','houseDecoration','communityName','region','plate','buildYear','All_School',
                 'Transportation','All_hospital','All_mall','other_num']
for i in range(len(cluster_col_2)):
    for j in range(i+1,len(cluster_col_2)):
        data_df[cluster_col_2[i]+'_'+cluster_col_2[j]] = data_df.groupby(['cluster',
                                                                          cluster_col_2[i]])[cluster_col_2[j]].transform('mean')
        data_df[cluster_col_2[i]+'_'+cluster_col_2[j]].fillna(0,inplace=True)

## -----------------------------------------baseline---------------------------------

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
categorical_new = [col for col in data_df.columns if data_df[col].dtypes!='object']
data_df[categorical_new].nunique().sort_values(ascending=False)

area                              9768
communityName                     4035
com_area_mean                     3729
com_area_std                      3108
other_ratio                       1331
Transportation_ratio              1331
School_ratio                      1331
mall_ratio                        1331
school_ratio                      1329
pv                                 714
tradeMeanPrice                     707
totalTradeArea                     707
totalTradeMoney                    706
SecMeanArea                        706
uv                                 653
totalNewTradeMoney                 561
tradeNewMeanPrice                  560
totalNewTradeArea                  535
remainNewNum                       393
communityName_All_mall             340
tradeSecNum                        334
communityName_Transportation       329
communityName_All_School           319
communityName_other_num            316
communityName_All_hospital         303
buildYear_All_mall       

In [20]:
# from sklearn.preprocessing import LabelEncoder

# Label_list = ['plate','region','houseToward']
# for col_4 in Label_list:
#     data_df[col_4] = data_df[col_4].astype(str)
#     data_df[col_4] = LabelEncoder().fit_transform(data_df[col_4])
    
# Onehot_list = ['houseDecoration','rentType','houseFloor']
# for col_5 in Onehot_list:
#     data_df[col_5] = data_df[col_5].astype(str)
#     inter_col = pd.get_dummies(data_df[col_5])
#     data_df.drop(col_5,axis=1,inplace=True)
#     data_df = pd.concat([data_df,inter_col],axis=1)

In [21]:
print(data_df.shape)
print(train_df.shape)
print(test_df.shape)

(41676, 115)
(39207, 51)
(2469, 50)


In [22]:
Y = train_df['tradeMoney']
m = len(Y)
X = data_df.iloc[:m,:]
X_test = data_df.iloc[m:,:]

In [23]:
print(X.shape)
print(X_test.shape)

(39207, 115)
(2469, 115)


In [24]:
from sklearn.metrics import r2_score
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold,cross_val_score
import time

In [25]:
params = {
    'num_leaves': 31,
    'min_data_in_leaf': 20,
    'min_child_samples':20,
    'objective': 'regression',
    'learning_rate': 0.01,
    "boosting": "gbdt",
    "feature_fraction": 0.8,
    "bagging_freq": 1,
    "bagging_fraction": 0.85,
    "bagging_seed": 23,
    "metric": 'rmse',
    "lambda_l1": 0.2,
    "nthread": 4,
}

In [26]:
folds = KFold(n_splits=5,shuffle=True,random_state=2019)
oof_lgb = np.zeros(len(X))
pred_lgb = np.zeros(len(X_test))
start = time.time()
for fold_,(trn_idx,val_idx) in enumerate(folds.split(X.values,Y.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(X.iloc[trn_idx],label=Y.iloc[trn_idx])
    
    val_data = lgb.Dataset(X.iloc[val_idx],label=Y.iloc[val_idx])
    

    num_round = 10000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds = 200)
        
    oof_lgb[val_idx] = clf.predict(X.iloc[val_idx], num_iteration=clf.best_iteration)
    pred_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV Score: {:<8.5f}".format(r2_score( Y, oof_lgb)))

fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's rmse: 824.459	valid_1's rmse: 949.572
[1000]	training's rmse: 729.654	valid_1's rmse: 894.288
[1500]	training's rmse: 673.99	valid_1's rmse: 872.255
[2000]	training's rmse: 632.804	valid_1's rmse: 858.684
[2500]	training's rmse: 600.666	valid_1's rmse: 850.315
[3000]	training's rmse: 573.618	valid_1's rmse: 844.205
[3500]	training's rmse: 550.433	valid_1's rmse: 839.369
[4000]	training's rmse: 530.046	valid_1's rmse: 835.968
[4500]	training's rmse: 512.324	valid_1's rmse: 833.959
[5000]	training's rmse: 495.582	valid_1's rmse: 831.752
[5500]	training's rmse: 480.683	valid_1's rmse: 830.236
[6000]	training's rmse: 466.707	valid_1's rmse: 829.449
[6500]	training's rmse: 453.866	valid_1's rmse: 828.586
[7000]	training's rmse: 441.971	valid_1's rmse: 828.054
[7500]	training's rmse: 430.902	valid_1's rmse: 827.392
[8000]	training's rmse: 420.479	valid_1's rmse: 827.259
[8500]	training's rmse: 410.541	va

In [27]:
def online_score(pred):
    print("预测结果最大值：{},预测结果最小值：{}".format(pred.max(),pred.min()))
    # a榜测分
    conmbine1 = pd.read_csv("D:/DataAnalysis/2019未来高校AI挑战赛_城市-房产租金预测/sub_a_913.csv",engine = "python",header=None)
    score1 = r2_score(pred, conmbine1.values)
    print("对比913分数:{}".format(score1))

In [28]:
online_score(pred_lgb)

预测结果最大值：17097.799079001084,预测结果最小值：1152.9007407421789
对比913分数:0.9782714515371335
